In [1]:
import pickle
import pandas as pd
import numpy as np
import re
import itertools
import json

In [2]:
from eunjeon import Mecab
mecab = Mecab()

In [3]:
data_path = "C:/Users/msi/GitHub/JasoseoAI_project/3.data/"

In [12]:
with open(data_path+'JobKorea/JobKorea_processed.pickle', 'rb') as f:
    jobkorea = pickle.load(f)

In [13]:
jobkorea.columns

Index(['company_name', 'company_url', 'apply_info', 'applicant_info', 'score',
       'score_desc', 'url', 'Question', 'content', 'comment'],
      dtype='object')

In [14]:
jobkorea = jobkorea[['company_name', 'content', 'Question']]

In [16]:
jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub(" +","",x.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", "")))
jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub("[^가-힣a-z0-9]", "", x))

In [17]:
jobkorea.head(3)

,company_name,content,Question
0,경동나비엔,[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 ...,[자신의 가장 자랑스러운 성취 경험 또는 자신을 가장 힘들게 했던 실패 경험은 무엇...
2,포스코인터내셔널,[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도전...,"[성장과정 및 생활신조 등 자기소개를 기술하여 주시기 바랍니다. , 성격, 능력 관..."
3,하나은행,"[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력저는 어려서 배운 컴퓨터 강의...",[자기소개서]


In [18]:
len(jobkorea)

6631

In [19]:
company_names = set(jobkorea['company_name'].tolist())
len(company_names)

1513

In [20]:
groupby = jobkorea[['content', 'Question']].groupby(jobkorea['company_name'])

In [21]:
company_contents = dict()

In [22]:
for name, group in groupby:
    company_contents[name] = group 

In [23]:
len(company_contents)

1513

In [24]:
print(list(company_contents.keys())[1300])

한국일보사


In [25]:
company_contents['한국방송공사']['content'].tolist()[0]

['경영기획이란 주어진 자원을 효율적으로 활용하는 분야라고 생각합니다. 이러한 경영기획에서 필요한 역량은 탁월한 숫자감각과 원활한 소통이라고 생각합니다. 저는 세 가지 측면에서 이러한 역량을 갖춘 인재입니다.첫째, 통계지식을 바탕으로 숫자감각을 키웠습니다. 사회조사분석사 공부를 통해 조사방법론, 분석 도구의 특징, 분석 자료의 해석 등 통계에 대한 기초를 쌓았습니다. 또한, 학교에서 통계수업을 통해 분석 자료를 어떻게 해석할 것인가, 경영 의사결정에 어떻게 반영할 것인가 등을 훈련하였습니다. 이러한 공부를 통해 통계는 어떤 일에 대한 통찰력과 프레임을 제공해준다고 느꼈습니다. 또한, 객관적인 근거자료로써 활용할 수 있다는 것을 느꼈습니다. 하지만 다양한 분석 모델을 활용하여 프레임 자체에 갇혀서는 안 된다는 것을 느꼈고, 수치를 이해하고 데이터의 트렌드를 읽는 역량이 중요하다는 것을 배웠습니다.둘째, 조직에서 소통을 가장 중요하게 생각합니다. 팀 과제를 하면서 소통을 통해 조직 내 어려움을 극복하고 최고의 성과를 낸 경험이 있습니다. 한 팀원이 처음에는 자발적인 참여를 하다가 중간부터 의욕을 잃어 보이고 참여율이 저조했습니다. 그래서 팀 과제 진행을 중단하고 모든 팀원과 한강에서 치킨과 맥주를 먹는 자리를 마련하였습니다. 휴식하며 대화를 나누어 보니 모두가 다른 수업의 과제나 아르바이트 등으로 지쳐있는 상태라는 것을 알게 되었습니다. 이 자리를 통해 서로 공감하고 소통하면서 의지가 될 수 있었고, 모든 팀원이 다시 자발적인 협력을 통해 팀 과제의 추진력을 높일 수 있었습니다. 그 결과 팀 과제 1등의 성적 또한 거둘 수 있었습니다. 이러한 경험을 통해 조직에서 가장 중요한 것은 소통이라고 느꼈습니다. 소통은 자발적이고 유기적인 협력을 끌어낼 수 있다고 느꼈습니다.셋째, 다양한 분야를 연결하는 융합적 사고를 키우고 있습니다. OOOO 신문을 구독하며 사회 및 경제의 흐름을 읽고 있습니다. 또한, 각종 포럼, 콜로키엄, 콘퍼런스 등에 참가하면서 시대의 흐름을 읽고

In [26]:
for name, infos in company_contents.items():
    questions = list(itertools.chain(*infos.Question.tolist()))
    contents = list(itertools.chain(*infos.content.tolist()))
    if len(questions) != len(contents): break
    company_contents[name] = {'question' : questions, 'answer' : contents}

In [27]:
with open(data_path+'JobKorea/CompanyContents.json', 'w', encoding='utf-8') as f: # 회사-[질문:답변] => dict 형식
    json.dump(company_contents, f, indent="\t")

### 첫 문장 추출

In [24]:
first_contents = dict()

In [33]:
for name, infos in company_contents.items():
    answers = infos['answer']
    checks = []
    for a in answers:
        temp = a.split(".")
        check = []
        for t in temp:
            result = re.findall('^[^가-힣a-zA-Z0-9. ]', t)
            if result:
                check.append(t)
        checks.append(check)
    first_contents[name] = {
        'question' : infos['question'], 
        'answer' : infos['answer'],
        'first_answer' : checks
    }

In [35]:
first_contents['코레일네트웍스(주)']

{'question': ['지원분야에 관심을 가지게 된 계기는 무엇이며, 전문성 향상을 위해 어떤 노력을 했는지 서술해 주십시오.',
  '본인이 사회경험, 학교생활, 취미활동 등에서 주인의식을 가지고 문제를 해결하고자  최선을 다한 경험을 서술해 주십시오.',
  '대화를 통해 동료를 이해하고 구성원과의 화합을 이끌어낸 경험을 서술해 주십시오.',
  '매사에 능동적이며 맡은 바 업무를 끝까지 해결한 경험을 서술해 주십시오.',
  '지원분야에 관심을 가지게 된 계기는 무엇, 전문성향상을 위해 어떤노력을 했는지 서술',
  '사회경험/학교생활/취미에서 주인의식을 갖고 문제해결하고자 최선다한 경험[',
  '매사에 능동적이며 맡은 바 업무를 끝까지 해결한 경험을 서술'],
 'answer': ['사무직은 사람과 사람을 이어주는 코레일네트웍스의 프로세스를 총괄하는 업무를 수행합니다. 따라서 다른 직무에 비해 다양한 데이터를 활용하고, 종합적으로 파악하고 발빠르게 판단을 내리는 것이 중요합니다. 소비자, 그리고 통계에 관심을 두었던 저의 경험을 적극 활용할 수 있다고 판단하여 지원하였습니다.데이터베이스 마케팅 관련 수업을 다수 수강하며 기업의 입장에서 유용한 데이터를 파악하고, 이를 마케팅 전략으로 구현하는 과정을 연습하였습니다. 이후 2달간 빅데이터 전문가 양성 교육을 수료하였습니다. R 프로그래밍을 활용하여 수십만 건의 도서관 데이터 전처리 과정을 실습했습니다. 나아가, 실제 데이터를 활용하여 200만 원의 매출을 창출한 경험이 있습니다. 특정 기업의 고객 후기를 분석하여 관련 정보성 컨텐츠를 제작하였고, 이는 매출 창출로 이어졌습니다. 데이터로부터 시사점을 도출하고, 행동전략으로 구체화한 경험을 바탕으로, 최적의 마케팅 전략을 도출해내는 사원이 되겠습니다. ',
  '대학교 시절, 이벤트 사업을 진행하였습니다. 10명의 팀원은 꽃 수급, 꽃배달, 고객 관리 부서로 나누어 업무를 맡았습니다. 그러던 중, 꽃배달 팀원과 고객 관리 팀원 간의 실시간 연락이 이뤄지

In [36]:
with open(data_path+'JobKorea/CompanyFirstContents.json', 'w', encoding='utf-8') as f: # 회사-[질문:답변] => dict 형식
    json.dump(first_contents, f, indent="\t")